In [ ]:
import pandas as pd
import numpy as np
import datetime as dt
from datetime import date
import os
import requests
import json
from email.mime.text import MIMEText
import glob
from openpyxl import workbook, load_workbook

API_TOKEN  = r

#creat df from data stored in followup
def get_df(API_TOKEN, page):
    url = f'https://app.followupcrm.com/api/v1/file-data?page={page}'
    params = {
        'api_token': API_TOKEN,
        'per_page': 15
    }
    response = requests.get(url, params=params)
    if response.status_code == 200:
        json_data = response.json()
        df = pd.json_normalize(json_data['data'])
        return df, json_data
    else:
        print(f"Error {response.status_code}: {response.text}")
# create a new project in followup
def create_followup_project(API_TOKEN, name, status, est_name, bid_date):
    url = f'https://app.followupcrm.com/api/v1/file-data'
    params = {
        'api_token': API_TOKEN,
    }
    payload =  {
        'name': name,
        "external_id": None,
        "external_vendor": None,
        "sales_status": status,
        "estimator": est_name,
        "bid_date": bid_date

    }
    response = requests.post(url, params=params, data = payload)
    if response.status_code == 200:
        json_data = response.json()
        df = pd.json_normalize(json_data['data'])
    else:
        print(f"Error {response.status_code}: {response.text}")

# creates a dataframe of all jobs in followup(pulled from API)
df_list = []
json = get_df(API_TOKEN, 1)[1]
last_page = json['meta']['last_page']
for i in range(1,last_page+1):
    df_list.append(get_df(API_TOKEN, i)[0])
# combine dfs
full_df = pd.concat(df_list)
full_df['str'] = ''


# get data from most recent csv in j drive planner folder creating new projects in followup if the name in planner is not in followup
planner_folder_path = r"J:\2_Estimating Tools\planner_crm_vp\planner exports\*"
list_of_files = glob.glob(planner_folder_path) # * means all if need specific format then *.csv
latest_file = max(list_of_files, key=os.path.getctime)
planner_df = pd.read_excel(latest_file, skiprows=8)
# if a project isn;t asigned in asign column assumed to be a name or not going into follow up 
planner_projects = planner_df.dropna(subset='Assigned to')
# see if planner project name is not in follow up
followup_projects = list(full_df['name'])
missing_projects = planner_projects[~planner_projects['Name'].isin(followup_projects)]
for index, job in missing_projects.iterrows():
    pj_name, est_name, bid_date = job['Name'], job['Assigned to'], job['Finish']
    # create_followup_project(API_TOKEN, pj_name, "From Planner", est_name, bid_date)
    # print(pj_name, bid_date)    


In [128]:
# magic string for correct shhets
ec_sheet = ["2599999 (EC)",1]
ew_sheet = ["2599999", 0]
# Fill in phase code workbook with missing jobs 
phase_code_path = r"J:\2_Estimating Tools\planner_crm_vp\1_Estimating JC Job Phases  master.xlsx"
# ec_planner = planner_projects[planner_projects["EC"]=="Yes"][]
ec_phase_projects = list(pd.read_excel(phase_code_path, sheet_name=ec_sheet[0], skiprows=2).dropna()['Description'])
ew_phase_projects = list(pd.read_excel(phase_code_path, sheet_name=ew_sheet[0], skiprows=4).dropna()['Description'])
# df of planner projects to compare
ec_planner = planner_projects[planner_projects["EC"]=="Yes"]
ew_planner = planner_projects[planner_projects["EC"]=="No"]
# dfs of projects to add to phase workboob
missing_ec_phase_projects = ec_planner[~ec_planner['Name'].isin(ec_phase_projects)]
missing_ew_phase_projects = ew_planner[~ew_planner['Name'].isin(ew_phase_projects)]
# todats date 


# fuction to add projects to the phase workbook 
def add_items_phasewb(phase_code_path,data, sheet):
    date_field = date.today().strftime("%m/%d/%Y")
    book = load_workbook(phase_code_path)
    ws = book.worksheets[sheet[1]]
    for cell in ws['C']:
        if cell.value is not None:
            max_row_in_column = cell.row +1
    # print(max_row_in_column)
    for ind, job in data.iterrows():
        pj_name, client = job['Name'], job['Client']
        ws[f"C{max_row_in_column}"] = pj_name
        ws[f"D{max_row_in_column}"] = client
        ws[f"G{max_row_in_column}"] = date_field
        max_row_in_column = max_row_in_column + 1
        
    # print(max_row_in_column)
    book.save(filename=phase_code_path)
        # print()
    # save a backup
    backup_path = r"J:\2_Estimating Tools\planner_crm_vp\generated data\back_up\back_up"
    book.save(filename=f'{backup_path}{str(date.today())}.xlsx')

add_items_phasewb(phase_code_path, missing_ec_phase_projects, ec_sheet)
add_items_phasewb(phase_code_path, missing_ew_phase_projects, ew_sheet)



In [ ]:
from openpyxl import workbook, load_workbook

# magic string for correct shhets
ec_sheet = "2599999 (EC)"
ew_sheet = "2599999"

# Fill in phase code workbook with missing jobs 
phase_code_path = "J:\2_Estimating Tools\planner_crm_vp\generated data\1_Estimating JC Job Phases  (SAVE).xls"
# df of phase codes for ec and ew work 
ec_phase_projects = pd.read_excel(phase_code_path, sheet_name=ec_sheet, skiprows=2)
ew_phase_projects = pd.read_excel(phase_code_path, sheet_name=ew_sheet, skiprows=4)
# df of planner projects to compare
ec_planner = planner_projects[planner_projects["EC"]=="Yes"]
ew_planner = planner_projects[planner_projects["EC"]=="No"]
# dfs of projects to add to phase workboob
missing_ec_phase_projects = ""
missing_ew_phase_projects = ""
# todats date 
date_field = str(date.today())


def add_items(file_path, data, sheet):
    index = 1
    for ind, job in data.iterrows():
        pj_name, client, bid_date = job['Name'], job['Client'], job['Finish']
        index = index+1


add_items(phase_code_path, missing_ec_phase_projects, ec_sheet)
add_items(phase_code_path, missing_ew_phase_projects, ew_sheet)





